In [ ]:
from neo4j import GraphDatabase

class Neo4jService(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()      
    
    def crear_cabecera(self, tx, nombre):
        tx.run("CREATE (:Cabecera {nombre: $nombre})", nombre=nombre)
    
    def crear_candidato(self, tx, nombre):
        tx.run("CREATE (:Candidato {nombre: $nombre})", nombre=nombre)

    def crear_noticias(self, tx, nombre):
        tx.run("CREATE (:Noticias {nombre: $nombre})", nombre=nombre)
    
    def crear_links(self, tx, nombre):
        tx.run("CREATE (:Links {nombre: $nombre})", nombre=nombre)
        
    def crear_contenido(self, tx, nombre):
        tx.run("CREATE (:Contenido {nombre: $nombre})", nombre=nombre)
        
    def crear_titulo(self, tx, nombre):
        tx.run("CREATE (:Titulo {nombre: $nombre})", nombre=nombre)

    def crear_fecha(self, tx, nombre):
        tx.run("CREATE (:Fecha {nombre: $nombre})", nombre=nombre)
        
    def crear_relacion_noticia(self, tx, nombre_noticias, nombre_noticia):
        tx.run("MATCH (a:Noticias {nombre: $nombre_noticias}) "
               "MATCH (b:Contenido {nombre: $nombre_noticia}) "
               "MERGE (a)-[:Noticias]->(b)",
               nombre_noticias=nombre_noticias, nombre_noticia=nombre_noticia)
    
    def crear_relacion_cabeceras(self, tx, nombre_cabecera, nombre_titulo):
        tx.run("MATCH (a:Cabecera {nombre: $nombre_cabecera}) "
               "MATCH (b:Titulo {nombre: $nombre_titulo}) "
               "MERGE (a)-[:TituloNoticia]->(b)",
               nombre_cabecera=nombre_cabecera, nombre_titulo=nombre_titulo)
    
    def crear_relacion_fecha(self, tx, nombre_links, nombre_fecha):
        tx.run("MATCH (a:Links {nombre: $nombre_links}) "
               "MATCH (b:Fecha {nombre: $nombre_fecha}) "
               "MERGE (a)-[:Fecha]->(b)",
               nombre_links=nombre_links, nombre_fecha=nombre_fecha)

    #relacion de candidato   cabecera links noticias 
    def crear_relacion_candidato_titulo(self, tx, nombre_candidato, nombre_titulo):
        tx.run("MATCH (a:Candidato {nombre: $nombre_candidato}) "
               "MATCH (b:Cabecera {nombre: $nombre_titulo}) "
               "MERGE (a)-[:Candidato_Titulo]->(b)",
               nombre_candidato=nombre_candidato, nombre_titulo=nombre_titulo)
        
    def crear_relacion_candidato_fecha(self, tx, nombre_candidato, nombre_fecha):
        tx.run("MATCH (a:Candidato {nombre: $nombre_candidato}) "
               "MATCH (b:Links {nombre: $nombre_fecha}) "
               "MERGE (a)-[:Candidato_Fecha]->(b)",
               nombre_candidato=nombre_candidato, nombre_fecha=nombre_fecha)
    
    def crear_relacion_candidato_contenido(self, tx, nombre_candidato, nombre_noticias):
        tx.run("MATCH (a:Candidato {nombre: $nombre_candidato}) "
               "MATCH (b:Noticias {nombre: $nombre_noticias}) "
               "MERGE (a)-[:Candidato_Noticias]->(b)",
               nombre_candidato=nombre_candidato, nombre_noticias=nombre_noticias)
        
print("ejecucion correcta")

In [ ]:
import requests
from bs4 import BeautifulSoup

neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'examen')
with neo4j._driver.session() as session:

    session.write_transaction(neo4j.crear_candidato , "Candidato")
    session.write_transaction(neo4j.crear_cabecera , "Titulos")
    session.write_transaction(neo4j.crear_links , "Fecha")
    session.write_transaction(neo4j.crear_noticias , "Noticias")
            
with open("C:/Users/esteb/Desktop/busqueda.txt","r") as archivo:
        
    for linea in archivo:
        print(linea)

        i=0
        a=0
        e=0
        m=0

        page = requests.get(linea)

        soup = BeautifulSoup(page.content,'html.parser')
        #print(soup.prettify())


        #result =soup.find_all("a href")
        #titulos
        titulos =soup.find_all("div",{"class":"BNeawe vvjwJb AP7Wnd"})
        #Contenido
        contenidohtml =soup.find_all("div",{"class":"BNeawe s3v9rd AP7Wnd"})
        #Fecha
        fecha =soup.find_all("span",{"class":"r0bn4c rQMQod"})
        
        print("resultado de la busqueda links ")
        a=0

        tituloNeo=list()
        for i in titulos:
            tituloNeo.append(i.text)

        descripcion=list()
        for i in contenidohtml:
            descripcion.append(i.text)
            #print(link)
            
#descripcion filtrada
        descripcionAux=list()    
        for elemento in descripcion:
            if((a%2)==0):
                descripcionAux.append(descripcion[a])
            a=a+1
            

        fechaNeo=list()
        for i in fecha:
            fechaNeo.append(i.text)

        
            
        print("Titulo")
        for elemento in tituloNeo:
            session.write_transaction(neo4j.crear_titulo , elemento)
            print(elemento)
            
        #Descripcion print("descripcion")
        for elemento1 in descripcionAux:
            session.write_transaction(neo4j.crear_contenido , elemento1)
            print(elemento1)
        
        
        print("fecha")
        for elemento2 in fechaNeo:
            session.write_transaction(neo4j.crear_fecha , elemento2)
            print(elemento2)
            

#relaciones
        #relacion de descripcion
        for total in descripcionAux:
            session.write_transaction(neo4j.crear_relacion_noticia, "Noticias", total)  
            
        #relacion de Fecha               
        for date in fechaNeo:
            session.write_transaction(neo4j.crear_relacion_fecha, "Fecha", date)  
        
        #relacion de Titulo  
        for titul in tituloNeo:
            session.write_transaction(neo4j.crear_relacion_cabeceras, "Titulos", titul) 
            
            
session.write_transaction(neo4j.crear_relacion_candidato_titulo, "Candidato", "Titulos")
session.write_transaction(neo4j.crear_relacion_candidato_contenido, "Candidato", "Noticias")
session.write_transaction(neo4j.crear_relacion_candidato_fecha, "Candidato", "Fecha")
print("fin del proceso")